In [1]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
import json
import requests
from IPython.display import JSON

In [7]:
url = "https://ionbeam-dev.ecmwf.int/api/v1/"

In [11]:
# Load in the ECMWF token 
with open(Path("~/.ecmwfapirc").expanduser(), "r") as f:
    api_creds = json.load(f)

print("Checking API credentials")
r = requests.get(f"https://api.ecmwf.int/v1/who-am-i?token={api_creds['key']}")
if r.status_code == 403: print("Your credentials are either wrong or have expired and need to be renewed at https://api.ecmwf.int/v1/key/")
r.raise_for_status()

# Make a session object the requests library can reuse connections
session = requests.Session()
session.headers["Authorization"] = f"Bearer {api_creds['key']}"

Checking API credentials


In [9]:
stations = session.get(url + "stations").json()
print(f"{len(stations) = }")

len(stations) = 3853


In [12]:
def map_to_simple_author(authors):
    lls = ["Bologna", "Genova", "Jerusalem", "LLWA", "Barcelona", "Dublin", "CIMA", "Amsterdam"]
    for ll in lls:
        for a in authors:
            if "gert-jan" in a["name"].lower():
                return "Amsterdam", a["name"]
            if ll.lower() in a["name"].lower():
                return ll, a["name"]
    return "/".join(a["name"] for a in authors), None

df = pd.DataFrame.from_records(dict(
    id= s["external_id"],
    platform= s["platform"],
    location= s["location_point"],
    start= s["start_time"],
    stop= s["stop_time"],
    author = map_to_simple_author(s["authors"])[0],
    )
    for s in stations)

geo_df = gpd.GeoDataFrame(df, geometry=gpd.GeoSeries.from_wkt(df["location"]), crs=4326)

geo_df.explore(column = "author")